# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
from imageio import imread
from skimage.transform import resize as imresize

In [2]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize as imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [14]:
train_doc = np.random.permutation(open('/opt/bucketdata/Users/Devashish/Networks/CNN/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/opt/bucketdata/Users/Devashish/Networks/CNN/Project_data/val.csv').readlines())
batch_size = 50

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [19]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,18,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imresize(image,(100,100))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0]  /= 255.0
                    batch_data[folder,idx,:,:,1]  /= 255.0
                    batch_data[folder,idx,:,:,2]  /= 255.0
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        rem_seq_len = 0
        if((len(t)%batch_size)!=0):
            rem_seq_len = len(t)%batch_size
        
        for batch in range(rem_seq_len): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,18,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imresize(image,(100,100))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0]  /= 255.0
                    batch_data[folder,idx,:,:,1]  /= 255.0
                    batch_data[folder,idx,:,:,2]  /= 255.0
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [20]:
curr_dt_time = datetime.datetime.now()
train_path = '/opt/bucketdata/Users/Devashish/Networks/CNN/Project_data/train'
val_path = '/opt/bucketdata/Users/Devashish/Networks/CNN/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [42]:
from keras.models import Sequential, Model
from keras.layers import *
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import optimizers

Input_shape = (18, 100, 100, 3)
filtersize = (3,3,3)

model = Sequential()
model.add(Conv3D(16, filtersize, padding='same', input_shape=Input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(32, filtersize, padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, filtersize, padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, filtersize, padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(5,activation='softmax'))


# model = Sequential()
# model.add(Conv3D(32, (2,2,2), padding='same',
#                  input_shape=Input_shape))
# #model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=sample_shape))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Conv3D(32, (2,2,2)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling3D(pool_size=(2, 2,2)))
# model.add(Dropout(0.5))

# model.add(Conv3D(64, (2,2,2), padding='same'))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(Conv3D(128, (2,2,2)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling3D(pool_size=(2, 2,2)))
# model.add(Dropout(0.5))
# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(5))
# model.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [43]:
optimiser = 'adam'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_17 (Conv3D)           (None, 18, 100, 100, 16)  1312      
_________________________________________________________________
activation_17 (Activation)   (None, 18, 100, 100, 16)  0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 18, 100, 100, 16)  64        
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 9, 50, 50, 16)     0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 9, 50, 50, 32)     13856     
_________________________________________________________________
activation_18 (Activation)   (None, 9, 50, 50, 32)     0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 9, 50, 50, 32)    

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [44]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
# img,label = next(train_generator)
# print(img.shape)

In [45]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)


filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 3,
                       verbose = 0, mode = "auto", epsilon = 1e-04, cooldown = 0,
                       min_lr = 0)
        
early_stop = EarlyStopping(monitor = "val_loss", patience=9, restore_best_weights =True, verbose=1)

callbacks_list = [checkpoint, LR, early_stop]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [46]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/50
Source path =  /opt/bucketdata/Users/Devashish/Networks/CNN/Project_data/val ; batch size = 50
Source path =  /opt/bucketdata/Users/Devashish/Networks/CNN/Project_data/train ; batch size = 50
14/14 [==============================] - 157s 11s/step - loss: 1.6096 - categorical_accuracy: 0.1857 - val_loss: 1.6077 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2021-11-2111_28_05.169274/model-00001-1.60963-0.18571-1.60766-0.23000.h5
Epoch 2/50
14/14 [==============================] - 123s 9s/step - loss: 1.6090 - categorical_accuracy: 0.2057 - val_loss: 1.6064 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2021-11-2111_28_05.169274/model-00002-1.60905-0.20571-1.60640-0.23000.h5
Epoch 3/50
10/14 [====================>.........] - ETA: 31s - loss: 1.6083 - categorical_accuracy: 0.2220